<a href="https://colab.research.google.com/github/MarwaRayen/Smart-Irrigation-System/blob/main/ClassificationRiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importation des librairies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Chargement des données

In [ ]:
# Charger le jeu de données
data = pd.read_excel("arachides+nouveau_riz.xlsx")

# Afficher les premières lignes du jeu de données pour comprendre sa structure
print(data.head())

# Afficher les colonnes pour vérifier leurs noms
print(data.columns)


   CropType  CropDays  Soil Moisture  Soil Temperature  Temperature  Humidity  \
0         1         1            630                19           33        52   
1         1         2            460                20           31        50   
2         1         3            230                22           33        48   
3         1         4            140                24           28        62   
4         1         5            720                18           28        63   

   Irrigation(Y/N)  Unnamed: 7  Unnamed: 8   Unnamed: 9 Unnamed: 10  
0                1         NaN         NaN          NaN         NaN  
1                0         NaN         NaN          NaN         NaN  
2                0         NaN         NaN     CropType        Code  
3                0         NaN         NaN        Paddy           1  
4                1         NaN         NaN  Ground Nuts           2  
Index(['CropType', 'CropDays', 'Soil Moisture', 'Soil Temperature',
       'Temperature', 'Hu

#Pré-traitement du dataset

**Nous supprimons ici les colonnes qui ne sont pas nécessaires pour notre analyse afin de simplifier le jeu de données**

In [ ]:
# Supprimer les colonnes inutiles du DataFrame
data = data.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'])

# Afficher le DataFrame après la suppression des colonnes pour s'assurer que c'est correct
print(data.head())


   CropType  CropDays  Soil Moisture  Soil Temperature  Temperature  Humidity  \
0         1         1            630                19           33        52   
1         1         2            460                20           31        50   
2         1         3            230                22           33        48   
3         1         4            140                24           28        62   
4         1         5            720                18           28        63   

   Irrigation(Y/N)  
0                1  
1                0  
2                0  
3                0  
4                1  


**vérifier la présence de valeurs manquantes dans les données et de les supprimer pour garantir l'intégrité du jeu de données.**

In [ ]:
# Vérifier s'il y a des valeurs manquantes
print(data.isnull().sum())

# Supprimer les lignes avec des valeurs manquantes (le cas échéant)
data = data.dropna()

# S'assurer qu'il n'y a plus de valeurs manquantes
print(data.isnull().sum())


CropType            0
CropDays            0
Soil Moisture       0
Soil Temperature    0
Temperature         0
Humidity            0
Irrigation(Y/N)     0
dtype: int64
CropType            0
CropDays            0
Soil Moisture       0
Soil Temperature    0
Temperature         0
Humidity            0
Irrigation(Y/N)     0
dtype: int64


**Nous convertissons ici la variable cible Irrigation(Y/N) en entier pour qu'elle soit utilisable par le modele de classification**

In [ ]:
# Encoder la variable cible
data['Irrigation(Y/N)'] = data['Irrigation(Y/N)'].astype(int)


In [ ]:
# Séparer les caractéristiques (features) et la variable cible (target)
X = data.drop(columns=['Irrigation(Y/N)'])
y = data['Irrigation(Y/N)']

# afficher quelques lignes
print(X.head())
print(y.head())


   CropType  CropDays  Soil Moisture  Soil Temperature  Temperature  Humidity
0         1         1            630                19           33        52
1         1         2            460                20           31        50
2         1         3            230                22           33        48
3         1         4            140                24           28        62
4         1         5            720                18           28        63
0    1
1    0
2    0
3    0
4    1
Name: Irrigation(Y/N), dtype: int64


**La normalisation des données permet de s'assurer que toutes les caractéristiques ont des échelles similaires**

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normaliser les données pour améliorer la performance des modèles de machine learning
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


#Entrainement du modele
Nous utilisons ici un modèle de régression logistique pour classifier si un champ a besoin d'être irrigué ou non. Après l'entraînement, nous évaluons la performance du modèle en calculant la précision et en générant un rapport de classification.

In [ ]:
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Définir la grille des paramètres pour la régression logistique
param_grid = {
    'penalty': ['l1', 'l2'],  # Types de régularisation à tester
    'C': [0.01, 0.1, 1, 10, 100],  # Différents coefficients de régularisation
    'solver': ['liblinear']  # Algorithme d'optimisation à utiliser
}

# Initialiser le modèle de régression logistique
model = LogisticRegression(random_state=42)

# Initialiser la recherche par grille avec validation croisée
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Ajuster la recherche par grille aux données d'entraînement
grid_search.fit(X_train, y_train)

# Obtenir les meilleurs paramètres et le meilleur score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Meilleurs paramètres : {best_params}")


Meilleurs paramètres : {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Meilleur score de validation croisée : 0.9333333333333332


#Utilisation du CROSS VALIDATION

Elle consiste à diviser les données d'entraînement en plusieurs sous-ensembles, et à entraîner et évaluer le modèle sur ces différents sous-ensembles de façon itérative

In [ ]:
from sklearn.model_selection import cross_val_score

# Effectuer une validation croisée sur le meilleur modèle
best_model = grid_search.best_estimator_
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)

# Afficher les scores de validation croisée
print("Scores de validation croisée :", cv_scores)
print("Score moyen de validation croisée :", cv_scores.mean())

Scores de validation croisée : [0.875      1.         0.91666667 0.91666667 0.95833333]
Score moyen de validation croisée : 0.9333333333333332


#Prédiction et évaluation

In [ ]:
# Entraîner le meilleur modèle sur l'ensemble des données d'entraînement
best_model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Générer le rapport de classification
print(classification_report(y_test, y_pred))

# Calculer et afficher le score d'exactitude
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision sur l'ensemble de test (accuracy) : {accuracy}")


              precision    recall  f1-score   support

           0       1.00      0.96      0.98        26
           1       0.80      1.00      0.89         4

    accuracy                           0.97        30
   macro avg       0.90      0.98      0.93        30
weighted avg       0.97      0.97      0.97        30

Précision sur l'ensemble de test (accuracy) : 0.9666666666666667


les résultats des scores de validation croisée montrent que le modèle a une performance élevée et cohérente, avec une précision moyenne de 96.66%. Cela signifie que le modèle est bien formé et devrait être capable de généraliser efficacement à de nouvelles données


#Ensembling

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

# Définir les modeles
log_reg = LogisticRegression(C=0.1, penalty='l2', solver='liblinear')
svm = SVC(kernel='linear', probability=True)

# Initialiser the voting classifier
ensemble_model = VotingClassifier(estimators=[('lr', log_reg), ('svm', svm)], voting='soft')

# Entrainer l'ensemble de modeles
ensemble_model.fit(X_train, y_train)

# Evaluaer l'ensemble de modeles
ensemble_pred = ensemble_model.predict(X_test)


print(classification_report(y_test, ensemble_pred))

# Calculer l'accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_pred)
print(f"Ensemble model accuracy: {ensemble_accuracy}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00         4

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Ensemble model accuracy: 1.0
